In [1]:
# Importing required libraries
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from textblob import Word
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
!pip install transformers

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
td=pd.read_csv("/content/drive/MyDrive/twitter sentiment analysis  /Dataset/twitter_training.csv",names=["id","company","labels","text"])
vd=pd.read_csv("/content/drive/MyDrive/twitter sentiment analysis  /Dataset/twitter_validation.csv",names=["id","company","labels","text"])

In [4]:
df = pd.concat([td,vd])

In [5]:
df.shape

(75682, 4)

In [6]:
df.head()

,id,company,labels,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [7]:
del df["id"]
del df["company"]

In [8]:
df.isnull().sum()

labels      0
text      686
dtype: int64

In [9]:
df=df.dropna()

In [10]:
df.isnull().sum()

labels    0
text      0
dtype: int64

In [11]:
df.duplicated().sum()

4744

In [12]:
df= df.drop_duplicates()

In [13]:
df = df[df['labels'] != "Irrelevant"]

In [14]:
df['labels'].value_counts()

Negative    21329
Positive    19271
Neutral     17281
Name: labels, dtype: int64

In [15]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [16]:
#Removing stop-words and converting words to lemma

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

def stop_wrds_lemma_convert(sentence):
    tokens = [w for w in sentence.split() if not w in stop_words] #stopwords removal
    newString=''
    for i in tokens:
        newString=newString+lemmatizer.lemmatize(i)+' '    #converting words to lemma
    return newString.strip()

In [17]:
def data_cleaning(x):
  df['tweet_clean']=x.apply(lambda x: " ".join(x.lower() for x in x.split()))#Converting them to lowercase
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: re.sub(r'@[A-Za-z0-9]+','',x))#Remove user mentions from the tweet
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: re.sub(r'[0-9]+','',x))
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: re.sub(r'http\S+','',x))
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: re.sub('#','',x))#Remove hashtags from the tweet
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in x.split(" ")]))#Remove contractions
  df['tweet_clean']=df['tweet_clean'].str.replace('[^\w\s]','')#Removing Puntuation
  df['tweet_clean']=df['tweet_clean'].apply(lambda x: stop_wrds_lemma_convert(x))
  return df['tweet_clean']

In [18]:
data_cleaning(df['text'])

<ipython-input-17-362836a89187>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tweet_clean']=df['tweet_clean'].str.replace('[^\w\s]','')#Removing Puntuation


0                           im getting borderland murder
1                                     coming border kill
2                             im getting borderland kill
3                            im coming borderland murder
4                           im getting borderland murder
                             ...                        
984    noticed streamer watch playing game battlefiel...
985    __man playing red dead redemption oh shit bear...
988    suikoden alex kidd miracle world persona soul ...
989    thank matching fund home depot rw payment gene...
991    late night stream boy come watch warzone run s...
Name: tweet_clean, Length: 57881, dtype: object

In [19]:
df.drop(columns=['text'],axis=0,inplace=True)

In [20]:
df

,labels,tweet_clean
0,Positive,im getting borderland murder
1,Positive,coming border kill
2,Positive,im getting borderland kill
3,Positive,im coming borderland murder
4,Positive,im getting borderland murder
...,...,...
984,Negative,noticed streamer watch playing game battlefiel...
985,Neutral,__man playing red dead redemption oh shit bear...
988,Neutral,suikoden alex kidd miracle world persona soul ...
989,Positive,thank matching fund home depot rw payment gene...


In [21]:
X=list(df['tweet_clean'])

In [22]:
y=list(df['labels'])

In [23]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('google/bert_uncased_L-8_H-512_A-8')
model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/twitter sentiment analysis  /Model")

In [24]:
model = model.to('cuda')

In [25]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)

In [26]:
def predict(text):
  inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt',max_length=512).to('cuda')
  outputs = model(**inputs)
  predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
  predictions = predictions.cpu().detach().numpy()
  if(np.argmax(predictions) == 0):
    return "Negative"
  elif (np.argmax(predictions) == 1):
    return "Positive"
  else:
    return "Neutral"

In [28]:
from tqdm import tqdm

predict_label = []
for sentence in tqdm(X_val):
  predict_label.append(predict(sentence))

100%|██████████| 11577/11577 [02:27<00:00, 78.62it/s]


In [29]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_val, predict_label))

print(confusion_matrix(y_val, predict_label))

              precision    recall  f1-score   support

    Negative       0.97      0.99      0.98      4266
     Neutral       0.99      0.97      0.98      3456
    Positive       0.98      0.98      0.98      3855

    accuracy                           0.98     11577
   macro avg       0.98      0.98      0.98     11577
weighted avg       0.98      0.98      0.98     11577

[[4211   19   36]
 [  53 3367   36]
 [  56   19 3780]]


In [30]:
print('Accuracy Score',accuracy_score(y_val, predict_label)*100,'%')
print('Precision Macro Score',precision_score(y_val, predict_label,average = 'macro')*100,'%')
print('Recall_Score',recall_score(y_val, predict_label, average = 'macro')*100,'%')
print('F1_Score',f1_score(y_val, predict_label, average = 'macro')*100,'%')

Accuracy Score 98.1083182171547 %
Precision Macro Score 98.16389569987734 %
Recall_Score 98.06332642639931 %
F1_Score 98.11050491427191 %
